In [1]:
from scraper import number_companies, valuetoday_usa_scraper, df_cleaning

In [15]:
dataframe = valuetoday_usa_scraper(21)
final_df = df_cleaning(dataframe)

IndexError: list index out of range

In [11]:
final_df

,Company Name,Company Business,Market Value,Annual Revenue,Operating Income,Net Income,Assets,Liabilities
0,APPLE,Technology,2256000000000,274510000000,66280000000,274510000000,317344000000,245062000000
1,MICROSOFT CORPORATION,Technology,1682000000000,143000000000,52950000000,143000000000,301311000000,183007000000
2,ALPHABET,Technology,1185000000000,182520000000,41220000000,182520000000,299243000000,86320000000
3,AMAZON.COM,eCommerce,1634000000000,386060000000,22890000000,386060000000,110908000000,184586000000
4,FACEBOOK,Communication Services,778040000000,85960000000,32670000000,85960000000,139691000000,29244000000
5,BERKSHIRE HATHAWAY,Financial Services,543679999999,327220000000,0,327220000000,829946000000,410730000000
6,J P MORGAN CHASE & CO,Financial Services,387330000000,122900000000,0,122900000000,3386000000000,3106000000000
7,VISA,Financial Services,482170000000,21840000000,14090000000,21840000000,77884000000,42360000000
8,JOHNSON & JOHNSON,Healthcare,414310000000,82580000000,19730000000,82580000000,158380000000,95402000000
9,WALMART,Retail,407840000000,523960000000,22540000000,523960000000,237382000000,162072000000


In [16]:
import requests
import re
import pandas as pd
import warnings
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

In [31]:
ua = UserAgent()
header = {'User-Agent':str(ua.chrome)}
url = "https://www.value.today/headquarters/united-states-america-usa?page=2"

source = requests.get(url, headers=header)
soup = BeautifulSoup(source.content, "html.parser")

# Create a list of divs with all required information.
info_blocks = soup.find_all("div", class_="node node--type-listed-companies node--view-mode-teaser ds-2col-stacked clearfix")

collected_names = []
collected_businesses = []
collected_values = []
collected_values_metrics = []
collected_revenues = []
collected_revenues_metrics = []
collected_op_incomes = []
collected_op_incomes_metrics = []
collected_net_incomes = []
collected_net_incomes_metrics = []
collected_assets = []
collected_assets_metrics = []
collected_liabilities = []
collected_liabilities_metrics = []

for item in info_blocks:

    name = item.find("h2").text.strip()
    collected_names.append(name)
    print(name)

    business = item.select("div.field--name-field-company-category-primary > div > div.field--item > a")[0].text.strip()
    collected_businesses.append(business)

    value = item.select("div.field--name-field-market-value-jan012021 > div.field--item")[0]["content"]
    collected_values.append(value)

    value_string = item.select("div.field--name-field-market-value-jan012021 > div.field--item")[0].text.strip()
    if "Billion" in value_string:
        collected_values_metrics.append("Billion")
    elif "Million" in value_string:
        collected_values_metrics.append("Million")

    revenue_string = item.select("div.field--name-field-annual-revenue > div.field--item")[0].text.strip()
    revenue = re.findall("\d+[.,]\d+", revenue_string)
    collected_revenues.append(revenue[0])

    if "Billion" in revenue_string:
        collected_revenues_metrics.append("Billion")
    elif "Million" in revenue_string:
        collected_revenues_metrics.append("Million")
        
    op_income_item = item.select("div.field--name-field-annual-operating-income > div.field--item")
    if len(op_income_item) == 0:
        collected_op_incomes.append(None)
    else:
        op_income_string = op_income_item[0].text.strip()
        op_income = re.findall("\d+[.,]\d+", op_income_string)
        collected_op_incomes.append(op_income[0])

    if len(op_income_item) == 0:
        collected_op_incomes_metrics.append(None)
    elif "Billion" in op_income_string:
        collected_op_incomes_metrics.append("Billion")
    elif "Million" in op_income_string:
        collected_op_incomes_metrics.append("Million")

    net_income_string = item.select("div.field--name-field-annual-revenue > div.field--item")[0].text.strip()
    net_income = re.findall("\d+[.,]\d+", net_income_string)
    collected_net_incomes.append(net_income[0])

    if "Billion" in net_income_string:
        collected_net_incomes_metrics.append("Billion")
    elif "Million" in net_income_string:
        collected_net_incomes_metrics.append("Million")

    assets_string = item.select("div.field--name-field-total-assets > div.field--item")[0].text.strip()
    assets = re.findall("\d+[.,]\d+", assets_string)
    collected_assets.append(assets[0])

    if "Billion" in assets_string:
        collected_assets_metrics.append("Billion")
    elif "Million" in assets_string:
        collected_assets_metrics.append("Million")

    liabilities_string = item.select("div.field--name-field-total-liabilities- > div.field--item")[0].text.strip()
    liabilities = re.findall("\d+[.,]\d+", liabilities_string)
    collected_liabilities.append(liabilities[0])

    if "Billion" in liabilities_string:
        collected_liabilities_metrics.append("Billion")
    elif "Million" in liabilities_string:
        collected_liabilities_metrics.append("Million")

    df = pd.DataFrame({"Company Name": collected_names, "Company Business": collected_businesses, "Market Value": collected_values, 
                        "Value Metric": collected_values_metrics, "Annual Revenue": collected_revenues, 
                        "Revenue Metric": collected_revenues_metrics, "Operating Income": collected_op_incomes, 
                        "Op Income Metric": collected_op_incomes_metrics, "Net Income": collected_net_incomes, 
                        "Net Income Metric": collected_net_incomes_metrics, "Assets": collected_assets, 
                        "Assets Metric": collected_assets_metrics, "Liabilities": collected_liabilities, 
                        "Liabilities Metric": collected_liabilities_metrics})

COCA-COLA COMPANY


IndexError: list index out of range

In [39]:
number = info_blocks[3].select("div.field--name-field-annual-operating-income > div.field--item")[0].text

In [40]:
number.split()

['581', 'Million', 'USD', 'for', 'Year', 'ending', 'December-2020']